#### wg 5 assignment
# Anomaly Detection

(This code brances off of committ 29821ccffaca5817b3fe0b3ae2d972f8132b7999.)

(As an aside, I'm using python 3, not 2. Only change that had to be made was on the print statement at [*])

Imports

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

Read data.

In [2]:
header = pd.read_table("kddcup.names.txt", header=None)
att_types = pd.read_table("training_attack_types.txt", sep=" ", header=None)

tr_raw = pd.read_csv("kddcup.data_10_percent_corrected", header=None)
test_raw = pd.read_csv("kddcup_testdata.corrected", header=None)

Do the preprocessing.

In [3]:
def preprocess(dat):
    dat.columns = header[0]
    att_types.columns = ["attack", "type"]
    dat["type"] = np.nan
    for i in range(0, len(att_types["attack"])):
        dat.loc[dat["attack"] == att_types.loc[i,].attack, "type"] = att_types.loc[i,].type
    dat.type = dat.type.fillna("unlisted")
    dat.attack = dat.attack.astype('category')

    dat[dat.select_dtypes(include=['number']).columns] = preprocessing.scale(
        dat[dat.select_dtypes(include=['number']).columns])
    dat = pd.get_dummies(dat, columns=['protocol_type', 'service', 'flag'])

    return dat

In [4]:
tr = preprocess(tr_raw)
test = preprocess(test_raw)  # actual test data (but labeled--don't cheat!)

/home/valentin/anaconda2/envs/DS-CCMLWI-wg5/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Prepare for classification.

In [5]:
# not occurring in training set but occuring in test set so we just add it
tr['service_icmp'] = 0
test['service_red_i'] = 0
test['service_urh_i'] = 0

In [6]:
tr_labels = tr["type"].values
tr_features = tr.drop(["type", "attack"], axis=1).values

In [7]:
test_labels = test["type"].values
test_features = test.drop(["type", "attack"], axis=1).values

In [8]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
clf.fit(tr_features, tr_labels)
# TODO: is this correct? why don't we ever clf.predict(test_features)?
print(clf.score(test_features, test_labels)) # [*]

0.966704069395


## finding a better model
We can automate algorithm selection and hyperparameter tuning. Auto-sklearn is a drop-in replacement of sci-kit learn that can do a lot of this (see for instance http://www.kdnuggets.com/2017/01/current-state-automated-machine-learning.html)

Let's go!

### small example from the documentation first
(to get an idea)

In [1]:
import autosklearn

In [2]:
import autosklearn.classification
import sklearn.cross_validation
import sklearn.datasets
import sklearn.metrics

In [3]:
digits = sklearn.datasets.load_digits()

In [4]:
X = digits.data
y = digits.target

In [5]:
X_train, X_test, y_train, y_test = \
    sklearn.cross_validation.train_test_split(X, y, random_state=1)

In [8]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=1200)

In [9]:
automl.fit(X_train, y_train)

You are already timing task: index_run5
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
You are already timing task: index_run6
Process pynisher function call:
Traceback (most recent call last):
  File "/home/valentin/anaconda2/envs/DS-CCMLWI-wg5/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/valentin/anaconda2/envs/DS-CCMLWI-wg5/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/valentin/anaconda2/envs/DS-CCMLWI-wg5/lib/python3.5/site-packages/pynisher/limit_function_call.py", line 83, in subprocess_func
    return_value = (

[WARNING] [2017-03-22 18:00:46,313:intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2017-03-22 18:00:46,313:intensifier] Challenger was the same as the current incumbent; Skipping challenger


You are already timing task: index_run6


[WARNING] [2017-03-22 18:01:01,781:intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2017-03-22 18:01:01,781:intensifier] Challenger was the same as the current incumbent; Skipping challenger


In [10]:
y_hat = automl.predict(X_test)

In [11]:
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))

Accuracy score 0.986666666667


Wow, this takes long! No wonder... (but default is to terminate after 60 minutes, see https://automl.github.io/auto-sklearn/stable/api.html)

TODO: how big is that data set? Based on that, for how long do we have to expect our computation to take if it should go anywhere? How much time to allocate?

### now back to our own task

In [ ]:
autoclf = autosklearn.classification.AutoSklearnClassifier()
# the above line translates the following line to autosklearn:
# clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
autoclf.fit(tr_features, tr_labels)
# the above line translates the following line to autosklearn:
# clf.fit(tr_features, tr_labels)
y_hat_intdect = autoclf.predict(test_features)
# the above line corresponds to nothing.
print("Accuracy score", sklearn.metrics.accuracy_score(test_features, y_hat_intdect))
# the above line translates the following line to autosklearn:
# print(clf.score(test_features, test_labels)) # [*]